In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
movies=spark.read.options(header='True',inferschema=True).csv('/content/netflix_titles.csv')

In [4]:
movies.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|Septem

In [5]:
from pyspark.sql.functions import *

In [6]:
movies.groupBy('type').agg(count('*')).show()

+-------------+--------+
|         type|count(1)|
+-------------+--------+
|         null|       1|
|      TV Show|    2676|
|        Movie|    6131|
|William Wyler|       1|
+-------------+--------+



In [7]:
tv=movies.filter(movies.type=='TV Show')
tv.show()

+-------+-------+--------------------+-----------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|         director|                cast|       country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+-----------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|     s2|TV Show|       Blood & Water|             null|Ama Qamata, Khosi...|  South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|  Julien Leclercq|Sami Bouajila, Tr...|          null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|
|     s4|TV Show|Jailbirds New Orl...|             null|                null|   

In [8]:
tv.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [9]:
# from pyspark.sql.types import IntegerType
# tv = tv.withColumn("duration", tv["duration"].cast(IntegerType()))
# tv.show()
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [10]:
from pyspark.sql.types import*
def fun(s):
  return s[0]
fun_udf=udf(fun,StringType())

tv=tv.withColumn('Seas',fun_udf('duration').cast(IntegerType()))
tv.show()
# tv.printSchema()

+-------+-------+--------------------+-----------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+----+
|show_id|   type|               title|         director|                cast|       country|        date_added|release_year|rating| duration|           listed_in|         description|Seas|
+-------+-------+--------------------+-----------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+----+
|     s2|TV Show|       Blood & Water|             null|Ama Qamata, Khosi...|  South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|   2|
|     s3|TV Show|           Ganglands|  Julien Leclercq|Sami Bouajila, Tr...|          null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|   1|
|     s4|TV Show|Jailbirds New Orl...|             null

In [11]:
tv.filter(tv.Seas>2).show()

+-------+-------+--------------------+---------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+----+
|show_id|   type|               title|       director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|Seas|
+-------+-------+--------------------+---------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+----+
|     s9|TV Show|The Great British...|Andy Devonshire|Mel Giedroyc, Sue...|      United Kingdom|September 24, 2021|        2021| TV-14|9 Seasons|British TV Shows,...|A talented batch ...|   9|
|    s16|TV Show|   Dear White People|           null|Logan Browning, B...|       United States|September 22, 2021|        2021| TV-MA|4 Seasons|TV Comedies, TV D...|"Students of colo...|   4|
|    s22|TV Show|Resurrection: Ert.

In [12]:
film =movies.filter(movies.type=='Movie')
film.show()

+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|        date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|     s1|Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|  90 min|       Documentaries|As her father nea...|
|     s7|Movie|My Little Pony: A...|Robert Cullen, Jo...|Vanessa Hudgens, ...|                null|September 24, 2021|        2021|    PG|  91 min|Children & Family...|Equestria's divid...|
|     s8|Movie|             Sankofa|        Haile 

In [16]:
film.groupBy('rating').count().show()

+-----------------+-----+
|           rating|count|
+-----------------+-----+
| November 1, 2020|    1|
| Shavidee Trotter|    1|
|             TV-Y|  131|
|    Maury Chaykin|    1|
|             2019|    1|
|             2017|    1|
|               UR|    3|
|   Benn Northover|    1|
|             null|    3|
|               PG|  286|
|      Jide Kosoko|    1|
|            TV-MA| 2052|
|  Jowharah Jones"|    1|
|         TV-Y7-FV|    5|
|   Itziar Aizpuru|    1|
|           84 min|    1|
|               NR|   75|
|            TV-PG|  539|
|            NC-17|    3|
|           74 min|    1|
+-----------------+-----+
only showing top 20 rows



In [17]:
film.select('rating').distinct().show()

+-----------------+
|           rating|
+-----------------+
| November 1, 2020|
| Shavidee Trotter|
|             TV-Y|
|    Maury Chaykin|
|             2019|
|             2017|
|               UR|
|   Benn Northover|
|             null|
|               PG|
|      Jide Kosoko|
|            TV-MA|
|  Jowharah Jones"|
|         TV-Y7-FV|
|   Itziar Aizpuru|
|           84 min|
|               NR|
|            TV-PG|
|            NC-17|
|           74 min|
+-----------------+
only showing top 20 rows



In [26]:
film.na.fill(value='NR',subset=["rating"]).filter(film.rating=='NR').show()

+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|        date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|  s5972|Movie|            (T)ERROR|Lyric R. Cabral, ...|                null|       United States|     June 30, 2016|        2015|    NR|  84 min|       Documentaries|This real-life lo...|
|  s5988|Movie|          13 Cameras|      Victor Zarcoff|PJ McCabe, Briann...|       United States|   August 13, 2016|        2015|    NR|  90 min|Horror Movies, In...|Young parents-to-...|
|  s6016|Movie|         4th Man Out|      Andrew N

In [24]:
film.filter(film.rating=='NR')
film.show()

+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|        date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|     s1|Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|  90 min|       Documentaries|As her father nea...|
|     s7|Movie|My Little Pony: A...|Robert Cullen, Jo...|Vanessa Hudgens, ...|                null|September 24, 2021|        2021|    PG|  91 min|Children & Family...|Equestria's divid...|
|     s8|Movie|             Sankofa|        Haile 

In [27]:
film.groupBy('date_added').agg(count('*')).show()

+------------------+--------+
|        date_added|count(1)|
+------------------+--------+
|      May 21, 2021|       6|
|     March 2, 2021|       2|
|September 23, 2020|       2|
| September 8, 2020|       3|
|    April 14, 2020|       2|
|     June 22, 2019|       2|
|      May 30, 2017|       1|
|    April 29, 2016|       2|
|    March 25, 2016|       1|
|  October 27, 2015|       1|
|   January 1, 2008|       1|
|     March 2, 2017|       2|
|  October 31, 2015|       1|
|     June 23, 2021|       3|
|  November 1, 2020|      20|
|  February 9, 2020|       2|
| November 28, 2019|       2|
|   October 5, 2019|       1|
|September 20, 2019|       4|
| November 22, 2018|       1|
+------------------+--------+
only showing top 20 rows



In [34]:
country_counts = movies.groupBy('country').agg(count('*').alias('title_count'))
top_countries = country_counts.orderBy(col('title_count').desc())
top_countries.show(10)

+--------------+-----------+
|       country|title_count|
+--------------+-----------+
| United States|       2805|
|         India|        972|
|          null|        832|
|United Kingdom|        419|
|         Japan|        245|
|   South Korea|        199|
|        Canada|        181|
|         Spain|        145|
|        France|        123|
|        Mexico|        110|
+--------------+-----------+
only showing top 10 rows

